### requests
- select : 복수의 값을 찾아올 때, 유사리스트 형태로 값을 반환 -> 반복문 사용가능
- select_one : 단수의 값을 찾아올 때
- text : 크롤링해온 값을 문자만 남기고 싶을 때 사용
- strip() : 해당 문자 앞, 뒤 빈여백 혹은 이스케이프 시퀀스 제거하고자 할 때 사용
- .
- #
- ["속성명"] : ["data-price"]
- get_text(" ", strip=True)
- find : (태그를 기준) 단수의 값을 찾아올 때
- find_all : (태그를 기준) 복수의 값을 찾아올 때

In [39]:
import requests
from bs4 import BeautifulSoup

url = "https://crawlingtestsite-260122.netlify.app/"
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.encoding = "utf-8"

soup = BeautifulSoup(response.text, "html.parser")

# 상품명은 여러개가 존재
# 복수의 상품명은 공통적인 특징 : product-name
# 한번 크롤링 > 3개를 동시에 찾아올 수 있어야 함
# 1개의 변수에 1개 x 3개의 값을 포함할 수 있어야 함
# 리스트 형태로 가져올 수 있어야 함

# for tag in soup.select(".product-name") :
#     print(tag.text.strip())
#strip() : 문자열 객체

# names = [tag.text.strip() for tag in soup.select(".product-name")] # -> 리스트 컴프리핸션
# print(names)

# 상품명 / 카테코리명
# for card in soup.select(".product-card"):
#     name = card.select_one(".product-name").text.strip()
#     category = card.select_one(".category").text.strip()
#     print(f"{name} / {category}")

# 상품 : 가격
# for card in soup.select(".product-card"):
#     name = card.select_one(".product-name").text.strip()
#     price = int(card.select_one(".price")["data-price"])
#     # print(type(price))
#     print(f"{name} : {price:,}")

# NEW / HOT 배지가 있는 상품만 상품명을 크롤링 하기
# for card in soup.select(".product-card") :
#     badge = card.select_one(".badge")
#     if badge :
#         name = card.select_one(".product-name").text.strip()
#         print(f"{name} / {badge.text.strip()}")

# 5만원 이상의 상품의 이름만 크롤링 하기
# for card in soup.select(".product-card") :
#     price = int(card.select_one(".price")["data-price"])
#     if price >= 50000 :
#         name = card.select_one(".product-name").text.strip()
#         print(f"{name} : {price:,}")

# 특정 사이트에 접속해서 데이터 수집 + 재활용 (프로토타입)
# 수집한 상품별 상품명, 카테코리, 가격을 하나의 딕셔너리로 만들고, 
# 해당 딕셔너리가 하나의 리스트 안에 저장될 수 있도록 하세요.
"""
[ 
  (
     "name" : "..."
     "category" : "..."
     "price" : "..."
  ) ,
   {},
   {}
]
"""
items = []

for card in soup.select(".product-card") :
    price = int(card.select_one(".price")["data-price"])
    items.append({
        "name" : card.select_one(".product-name").text.strip(),
        "category" : card.select_one(".category").text.strip(),
        "price" : int(card.select_one(".price")["data-price"])
    })
print(items)


[{'name': '오프화이트 반팔 티셔츠', 'category': '의류', 'price': 89000}, {'name': '나이키 러닝화', 'category': '신발', 'price': 129000}, {'name': '가죽 카드 지갑', 'category': '잡화', 'price': 45000}]


In [43]:
from bs4 import BeautifulSoup

html = """
<p class="desc">
    여름용
    <strong>오프화이트</strong>
    반팔 티셔츠
<p/>
"""

soup = BeautifulSoup(html, "html.parser")
# desc = soup.select_one(".desc").text.strip() --> 앞 뒤쪽 영역의 빈 공백만 해결
desc = soup.select_one(".desc").get_text(" ", strip=True) # --> 내부 영역의 빈 공백까지도 영향
print(desc)

여름용 오프화이트 반팔 티셔츠


In [53]:
from bs4 import BeautifulSoup

html = """
<div class="product-card">
    <h2 class="product-name">나이키 러닝화</h2>
    <p class="category">신발</p>
    <p class="price" data-price="129000">129,000원</p>
</div>
"""

soup = BeautifulSoup(html, "html.parser")
# for tag in soup.select(".product-name") :
#     print(tag.text.strip())
# strip() : 문자열 객체

card = soup.find("div", class_="product-card") # 태그 중심
name = card.find("h2", class_="product-name").text.strip()
price = card.find("p", class_="price")["data-price"] # select_one --> 문서 위쪽부터 스캔 => 매칭되는 값이 나타나는 순간, 스캔 x
print(price)

129000


In [63]:
from bs4 import BeautifulSoup

html = """
<body id="body">
    <p class="cssstyle">오늘의 추천 상품입니다.</p>
    <p class="cssstyle">지금 구매하면 할인!</p>
    <p align="center">이 문구는 중앙정렬 안내입니다.</p>
    <div class="product-card">
        <h2 class="product-name">나이키 러닝화</h2>
        <p class="category">신발</p>
        <p class="price" data-price="129000">129,000원</p>
    </div>
</body>
"""
soup = BeautifulSoup(html, "html.parser")

# data = soup.find("p", class_="cssstyle")
# data = soup.find("p", "cssstyle") # class_ 생략가능
# data = soup.find("p", attrs={"align": "center"})
data = soup.find(id="body")
print(data.get_text(" ", strip=True))

# datas = soup.find_all("p", class_="cssstyle") # print(datas)
# for data in datas :
#     print(data.get_text(" ", strip=True))

# 1) select & select_one ----> 우선적으로 사용
# 2) find & find_all (* 일반적인 사이트는 대부분 div 남발)

오늘의 추천 상품입니다. 지금 구매하면 할인! 이 문구는 중앙정렬 안내입니다. 나이키 러닝화 신발 129,000원
